In [ ]:
import nonlinear_benchmarks
from nonlinear_benchmarks import error_metrics as metrics
import matplotlib.pyplot as plt
import torch
from model import StateSpaceSimulator

In [ ]:
import torch.nn as nn

class NeuralStateUpdate(nn.Module):

    def __init__(self, n_x, n_u, n_feat=32):
        super(NeuralStateUpdate, self).__init__()
        
        self.net = nn.Sequential(
            nn.Linear(n_x+n_u, n_feat),
            nn.Tanh(),
            nn.Linear(n_feat, n_x),
        )

        for m in self.net.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0, std=1e-2)
                nn.init.constant_(m.bias, val=0)
                    
    def forward(self, x, u):
        z = torch.cat((x, u), dim=-1)
        dx = self.net(z)
        return dx
    

class NeuralOutput(nn.Module):

    def __init__(self, n_x, n_y, n_feat=32):
        super(NeuralOutput, self).__init__()
        
        self.net = nn.Sequential(
            nn.Linear(n_x, n_feat),
            nn.Tanh(),
            nn.Linear(n_feat, n_y),
        )
                    
    def forward(self, x):
        y = self.net(x)
        return y

In [ ]:
# Load training results
ckpt = torch.load("ckpt_model1.pt")
cfg = ckpt["cfg"]

In [ ]:
# Instantiate model and load trained parameters
f_xu = NeuralStateUpdate(cfg.n_x, 1, n_feat=cfg.n_feat)
g_x = NeuralOutput(cfg.n_x, n_y=1, n_feat=cfg.n_feat)
model = StateSpaceSimulator(f_xu)
#x0 = torch.zeros((1, cfg.n_x))
x0 = ckpt["x0"] # according to the benchmark description, the test is initialized as the training...
f_xu.load_state_dict(ckpt["f_xu"])
g_x.load_state_dict(ckpt["g_x"])

In [ ]:
# Load test data
#test, _ = nonlinear_benchmarks.Cascaded_Tanks() # train performance
_, test = nonlinear_benchmarks.Cascaded_Tanks()
u_test, y_test = test
u_test = u_test.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)
ts = test.sampling_time

In [ ]:
# Load scalers
scaler_u = ckpt["scaler_u"]
scaler_y = ckpt["scaler_y"]

In [ ]:
# Scale u, simulate, and inverse scale result
ut = torch.tensor(scaler_u.transform(u_test)).unsqueeze(0).float()
with torch.no_grad():
    x_sim = model(x0, ut)
    y_sim = g_x(x_sim).squeeze(0)
    y_test_hat = y_sim.numpy() # output is the second state

y_test_hat = scaler_y.inverse_transform(y_test_hat)

In [ ]:
# Plot results
plt.figure()
plt.plot(y_test, "k")
plt.plot(y_test_hat, "b")
plt.plot(y_test_hat - y_test, "r")
plt.show()

In [ ]:
#mse = np.mean((y_test_hat - y_test)**2)
#rmse = np.sqrt(mse) * 1000
rmse = metrics.RMSE(y_test_hat, y_test)[0]
fit = metrics.fit_index(y_test_hat, y_test)[0]
print(f"{rmse=:.2f} V\n{fit=:.1f} %") 